[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/LLaVA-OneVision-jupyter/blob/main/LLaVA_OneVision_Qwen2_7b_si_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI
!git clone -b totoro4 https://github.com/camenduru/ComfyUI-LLaVA-OneVision /content/TotoroUI/custom_nodes/TotoroUI-LLaVA-OneVision
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.27 gradio==4.40.0 bitsandbytes

%cd /content/TotoroUI

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from nodes import load_custom_node
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_flux
from totoro import model_management
import gradio as gr

load_custom_node("/content/TotoroUI/custom_nodes/TotoroUI-LLaVA-OneVision")

DownloadAndLoadLLaVAOneVisionModel = NODE_CLASS_MAPPINGS["DownloadAndLoadLLaVAOneVisionModel"]()
LLaVA_OneVision_Run = NODE_CLASS_MAPPINGS["LLaVA_OneVision_Run"]()
LoadImage =  NODE_CLASS_MAPPINGS["LoadImage"]()

with torch.inference_mode():
    llava_model = DownloadAndLoadLLaVAOneVisionModel.loadmodel("lmms-lab/llava-onevision-qwen2-7b-si", "cuda", "nf4", "sdpa")[0]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

@torch.inference_mode()
def tag(image, tag_prompt):
    image = LoadImage.load_image(image)[0]
    text = LLaVA_OneVision_Run.run(image, llava_model, tag_prompt, 4096, True)[0]
    return text

with gr.Blocks(analytics_enabled=False) as demo:
    with gr.Row():
        with gr.Column():
            image = gr.Image(type='filepath')
        with gr.Column():
            tag_prompt = gr.Textbox(lines=3, interactive=True, value="prompt style", label="Tag Prompt")
            tag_button = gr.Button("Tag")
            positive_prompt = gr.Textbox(lines=3, interactive=True, value="", label="Image Prompt")

    tag_button.click(fn=tag, inputs=[image, tag_prompt], outputs=positive_prompt)

demo.queue().launch(inline=False, share=True, debug=True)